In [1]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

/Users/champagnepuppy/de-projects/Housing_Price_ML_E2E/.venv/bin/python
3.0.4
/Users/champagnepuppy/de-projects/Housing_Price_ML_E2E/.venv/lib/python3.11/site-packages/xgboost/__init__.py


In [2]:
# 1. Imports

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

/Users/champagnepuppy/de-projects/Housing_Price_ML_E2E/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 2. Load processed datasets

train_df = pd.read_csv("/Users/champagnepuppy/de-projects/Housing_Price_ML_E2E/data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("/Users/champagnepuppy/de-projects/Housing_Price_ML_E2E/data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (578878, 39)
Eval shape: (148697, 39)


In [4]:
# 3. Define Optuna objective function with MLflow

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [6]:
# 4. Run Optuna study with MLflow

# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("/Users/champagnepuppy/de-projects/Housing_Price_ML_E2E/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("Best params:", study.best_trial.params)

/Users/champagnepuppy/de-projects/Housing_Price_ML_E2E/.venv/lib/python3.11/site-packages/mlflow/tracking/_tracking_service/utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)
2025/12/15 16:04:17 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-12-15 16:04:17,604] A new study created in memory with name: no-name-8b4c371c-cf09-40d8-99f6-9b22c6025deb
[I 2025-12-15 16:04:36,333] Trial 0 finished with value: 72123.46210250052 and parameters: {'n_estimators': 688, 'max_depth': 9, 'learning_rate': 0.040744424071839286, 'subsample': 0.6313411422759175, 'colsample_bytree': 0.7761093443644255, 'min_child

Best params: {'n_estimators': 517, 'max_depth': 8, 'learning_rate': 0.06863181574594893, 'subsample': 0.8845671239765769, 'colsample_bytree': 0.5795323104673558, 'min_child_weight': 8, 'gamma': 1.4035516624939908, 'reg_alpha': 1.4470411791111491e-08, 'reg_lambda': 1.7704227852521917e-05}


In [7]:
# 5. Train final model with best params and log to MLflow

best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 32019.339325261702
RMSE: 71408.84412547745
R²: 0.9605850802192069


2025/12/15 16:12:31 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
